In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model



from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:


train_datagen = ImageDataGenerator(
    rescale=1./255,            # Rescale pixel values to [0, 1]
    rotation_range=20,         # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,     # Randomly shift images horizontally by up to 20% of width
    height_shift_range=0.2,    # Randomly shift images vertically by up to 20% of height
    shear_range=0.2,           # Apply random shear transformations
    zoom_range=0.2,            # Randomly zoom into images by up to 20%
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill in missing pixels with nearest pixel values
)

# Define ImageDataGenerator for validation data (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Set batch size
batch_size = 32

# Flow training images in batches using train_datagen
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/archive (9) - Copy - Copy/Data/train',                 # Directory containing training images
    target_size=(224, 224),    # Resize images to match input size of VGG16
    batch_size=batch_size,     # Batch size
    class_mode='categorical'   # Specify categorical labels
)

# Flow validation images in batches using validation_datagen
validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/archive (9) - Copy - Copy/Data/test',            # Directory containing validation images
    target_size=(224, 224),    # Resize images to match input size of VGG16
    batch_size=batch_size,     # Batch size
    class_mode='categorical'   # Specify categorical labels
)

Found 1437 images belonging to 3 classes.
Found 404 images belonging to 3 classes.


In [ ]:

# Load the pre-trained InceptionV3 model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:

# Add custom classification layers on top of the pre-trained layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(3, activation='softmax')(x)  # Assuming 3 classes


In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_480 (Conv2D)         (None, 127, 127, 32)         864       ['input_6[0][0]']             
                                                                                                  
 batch_normalization_470 (B  (None, 127, 127, 32)         96        ['conv2d_480[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_470 (Activation  (None, 127, 127, 32)         0         ['batch_normalization_47

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),  # Adjust the learning rate if needed
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
45/45 [==============================] - 348s 8s/step - loss: 0.7552 - accuracy: 0.7363 - val_loss: 0.3061 - val_accuracy: 0.8837
Epoch 2/10
45/45 [==============================] - 57s 1s/step - loss: 0.3431 - accuracy: 0.8636 - val_loss: 0.3394 - val_accuracy: 0.8564
Epoch 3/10
45/45 [==============================] - 57s 1s/step - loss: 0.3449 - accuracy: 0.8629 - val_loss: 0.2852 - val_accuracy: 0.8911
Epoch 4/10
45/45 [==============================] - 58s 1s/step - loss: 0.2654 - accuracy: 0.8970 - val_loss: 0.2406 - val_accuracy: 0.9158
Epoch 5/10
45/45 [==============================] - 58s 1s/step - loss: 0.2690 - accuracy: 0.8977 - val_loss: 0.2326 - val_accuracy: 0.9109
Epoch 6/10
45/45 [==============================] - 57s 1s/step - loss: 0.2527 - accuracy: 0.8977 - val_loss: 0.2430 - val_accuracy: 0.9010
Epoch 7/10
45/45 [==============================] - 57s 1s/step - loss: 0.2577 - accuracy: 0.9005 - val_loss: 0.2531 - val_accuracy: 0.9084
Epoch 8/10
45/45 [=

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()


NameError: name 'plt' is not defined

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)